%%HTML
<style>

.CodeMirror {
    width: 100vw;
}

.container {
    width: 69% !important;
}

.rendered_html {
  font-size:0.8em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}

</style>

### Pushbuttons

In [ ]:
from machine import Pin

button = Pin(27, Pin.IN, Pin.PULL_UP)

In [1]:
print(button.value())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'button' is not defined


In [ ]:
from time import sleep

while True:
    print(button.value())
    sleep(1)

## Interrupt handlers

ISR (interrupt service routines)

In [103]:
def interrupt(p):
    print("Button clicked!")
    
button.irq(trigger=Pin.IRQ_FALLING, handler=interrupt)

In [10]:
import machine
help(machine.Timer)

object <class 'Timer'> is of type type
  __del__ -- <function>
  deinit -- <function>
  init -- <function>
  value -- <function>
  ONE_SHOT -- 0
  PERIODIC -- 1


In [4]:
import micropython
micropython.alloc_emergency_exception_buf(100)

## OLED Display (SSD-1351)

![ssd1351](images/components/ssd1351.jpg)

## SPI (Serial Peripheral Interface Bus) - 1980s


![SPI](https://upload.wikimedia.org/wikipedia/commons/9/97/SPI_three_slaves_daisy_chained.svg)

# I2C (Inter-Integrated Circuit) - 1982

![I2C](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/I2C.svg/300px-I2C.svg.png)

- SPI: 
 - software (-1) and 
 - hardware (0, 1, 2, ..),  up to 80Mhz
- I2C: software (bit-banging)

In [11]:
from machine import Pin, SPI, ADC
from time import sleep
from ssd1351 import Display, color565

def setup_display():
    spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(18), sck=Pin(5))
    display = Display(spi, rst=Pin(26), dc=Pin(25), cs=Pin(4))
    return display

def cleanup_display(display):
    print('clearing display')
    display.clear()
    display.cleanup()

display = setup_display()

In [3]:
spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(18), sck=Pin(5))
display = Display(spi, rst=Pin(26), dc=Pin(25), cs=Pin(4))

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: SPI device already in use


In [4]:
display.clear()
display.cleanup()

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "ssd1351.py", line 137, in clear
  File "ssd1351.py", line 111, in block
  File "ssd1351.py", line 819, in write_cmd
OSError: transfer on deinitialized SPI


In [6]:
display.fill_rectangle(0, 0, 100, 100, color565(255, 255, 0))

In [5]:
from xglcd_font import XglcdFont
font = XglcdFont('fonts/Bally7x9.c', 7, 9)

display.draw_text(10, 20, "NDC 2018!", font, color565(0, 0, 255))

<p align="center">
    <img src="images/mine/ssd1351_text.jpg" width=500>
</p>

### Joystick 

<img align="left" src="images/components/joystick.jpg" width=300>


<img align="left" src="images/concepts/ADC.png" width=500>
- Attentuation level (voltage measurement range)
- Bit size (resolution)

In [ ]:
import time
t = time.time()
print(t)
time.sleep(1)
print(time.time()-t)

In [12]:
from machine import Pin, ADC
import time
sw = Pin(27, Pin.IN, Pin.PULL_UP)
adcl = ADC(Pin(32))
adcl.atten(ADC.ATTN_11DB)
adcr = ADC(Pin(39))
adcr.atten(ADC.ATTN_11DB) # change attentuation level 0-3.3V

color = color565(255, 0, 0)
while True:
    if sw.value() == 0:
        display.clear(color565(0, 0, 0))
        break
    x = 127 - (max(6, min(120, int(adcl.read()/32))))
    y = max(6, min(120, int(adcr.read()/32)))
    display.clear(color565(0, 0, 0))
    display.fill_circle(x, y, 6, color)
    sleep(0.01)

In [15]:
for i in range(30):
    display.draw_circle(64, 64, i, color565(i*4, 0, 0))

Consider adding irq on switch button to change color

### Microphone

<img align="left" src="images/components/max9814 mic.jpeg" width=300>

In [14]:
from machine import Pin, SPI, ADC
from time import sleep
mic = ADC(Pin(33))
mic.atten(ADC.ATTN_11DB)
print(mic.read())

1323


In [15]:
display.clear(color565(0, 0, 0))
sleep(0.5)
try:
    while True:
        if sw.value() == 0:
            display.clear(color565(0, 0, 0))
            break
        for i in range(127):
            val = int(mic.read())
            display.draw_vline(i%127, 0, min(max(int(val/15)-44,10),127), color565((i*2%255), 255-(i*2%255), 255-int(((i*2%255)/2))))
            sleep(0.01)
        display.clear(color565(0, 0, 0))
        gc.collect()
finally:
    display.clear(color565(0, 0, 0))
    gc.collect()

## Rotary encoder

<img align="left" src="images/components/encoder.jpg" width=300>

In [9]:
from time import sleep_ms
from encoder import Encoder
from machine import Pin

maxduty = const(122)
minduty = const(30)
e = Encoder(pin_clk=36, pin_dt=15, clicks=4, min_val=minduty, max_val=maxduty, start_val=77)
e_click = Pin(27, Pin.IN)

colors = [
    (255,0,0),
    (255, 100, 0),
    (255, 255, 0),
    (153, 255, 0),
    (0, 255, 255),
    (255, 51, 255),
    (0, 0, 255),
]
numcolors = len(colors)

val = e.value
try:
    while True:
        if sw.value() == 0:
            display.clear(color565(0, 0, 0))
            break
        if e.value != val:
            val = e.value
            #print(val)
            i = val % numcolors
            color = colors[i]
            display.clear(color565(*color))
            display.fill_polygon(sides=i+3, x0=64, y0=64, r=35, color=color565(255, 255, 255))
        sleep_ms(50)
finally:
    display.clear()
    display.cleanup()

display off


### MicroSD

from example:

spi = SPI(-1, 10000000, miso=Pin(19), mosi=Pin(18), sck=Pin(5))

from machine import Pin, SPI, ADC
from time import sleep
spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(18), sck=Pin(5))

In [7]:
import os, sdcard
from machine import SPI, Pin

spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(18), sck=Pin(5))
sd = sdcard.SDCard(spi, Pin(14))
os.mount(sd, '/sd')

In [10]:
with open('/sd/testfile.txt', 'w') as f:
    f.write('Hello NDC!')

In [11]:
print(os.listdir('sd'))

['testfile.txt']


In [ ]:
with open('/sd/testfile.txt', 'r') as f:
    print(f.read())
os.remove('/sd/testfile.txt')

In [ ]:
os.remove('/sd/testfile.py')